In [1]:
import os
import re
import glob
import xarray as xr
import rasterio
import rioxarray
import subprocess
import numpy as np
import pandas as pd
import netCDF4 as nc
import seaborn as sns
import multiprocessing as mp
import matplotlib.pyplot as plt

from collections import Counter
from osgeo import gdal, osr, ogr
from osgeo.gdal import InfoOptions
from rasterio.features import shapes
from geopy.geocoders import Nominatim
#from pyramids_gis.pyramids.dataset import Dataset
from numpy.lib.stride_tricks import sliding_window_view, as_strided


#%matplotlib inline
#%config InlineBackend.figure_format='retina'

In [2]:
path_GBIOMASS= "D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GBIOMASS.NC"
path_GLCT= "D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT.NC"
path_GFRAC = "D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC.NC"
path_GRAPC = "D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GRAPC.NC"

In [3]:
# gbiomass_5min_before = xr.open_dataset(path_GBIOMASS, engine="netcdf4", decode_times=True)

# gbiomass_5min_before.isnull()
# gbiomass_5min = gbiomass_5min_before.fillna(0.0)

# gbiomass_5min.isnull()

nc_gbiomass = nc.Dataset(path_GBIOMASS, mode='r')
gbiomass_5min_before = xr.open_dataset(xr.backends.NetCDF4DataStore(nc_gbiomass))
gbiomass_5min = gbiomass_5min_before.fillna(0.0)
gbiomass_5min

<xarray.Dataset>
Dimensions:    (time: 27, latitude: 2160, longitude: 4320, NBP: 9)
Coordinates:
  * NBP        (NBP) |S50 b'stems                                            ...
  * longitude  (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * latitude   (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * time       (time) datetime64[ns] 1970-01-01 1975-01-01 ... 2100-01-01
Data variables:
    GBIOMASS   (time, latitude, longitude, NBP) float32 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    modelname:    IMAGE_LPJ
    modelinfo:    image25/MODEL/trunk/model#2844,interface_imagelpj/trunk#170,#0
    conventions:  CF-1.6
    institution:  PBL netherlands Environmental Assessment Agency ( http//www...
    source:       image3.2,interface3.2,lpj3.2
    references:   http//www.pbl.nl/image
    disclaimer:   http//themasites.pbl.nl/models/image/index.php/IMAGE-rights

In [4]:
# %%
glct = xr.open_dataset(path_GLCT,decode_times=True)
glct

<xarray.Dataset>
Dimensions:    (longitude: 4320, latitude: 2160, time: 27)
Coordinates:
  * longitude  (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * latitude   (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * time       (time) datetime64[ns] 1970-01-01 1975-01-01 ... 2100-01-01
Data variables:
    GLCT       (time, latitude, longitude) float64 ...
Attributes:
    modelname:    IMAGE_LPJ
    modelinfo:    image25/MODEL/trunk/model#2844,interface_imagelpj/trunk#170,#0
    conventions:  CF-1.6
    institution:  PBL netherlands Environmental Assessment Agency ( http//www...
    source:       image3.2,interface3.2,lpj3.2
    references:   http//www.pbl.nl/image
    disclaimer:   http//themasites.pbl.nl/models/image/index.php/IMAGE-rights

In [5]:
# ### LUH2 states

path_LUH2= r"D:\kerja\asisten riset\vol\milkunC\achaidir\LUH2 2022\states.nc"
LUH2_states = xr.open_dataset(path_LUH2, engine="netcdf4", decode_times=False)

luh2_states_worldwide = LUH2_states.isel(time=slice(1000, 1173))
luh2_states_worldwide['time'] = pd.date_range(start="1850-01-01", end="2022-01-01", freq='YS')
luh2_states_worldwide

c:\Users\HEFRY ANESTI\AppData\Local\Programs\Python\Python39\lib\site-packages\xarray\coding\times.py:167: SerializationWarning: Ambiguous reference date string: 850-01-01 0:0:0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-01-01 0:0:0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


<xarray.Dataset>
Dimensions:  (time: 173, lon: 1440, lat: 720)
Coordinates:
  * time     (time) datetime64[ns] 1850-01-01 1851-01-01 ... 2022-01-01
  * lon      (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * lat      (lat) float64 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
Data variables: (12/14)
    primf    (time, lat, lon) float32 ...
    primn    (time, lat, lon) float32 ...
    secdf    (time, lat, lon) float32 ...
    secdn    (time, lat, lon) float32 ...
    urban    (time, lat, lon) float32 ...
    c3ann    (time, lat, lon) float32 ...
    ...       ...
    c4per    (time, lat, lon) float32 ...
    c3nfx    (time, lat, lon) float32 ...
    pastr    (time, lat, lon) float32 ...
    range    (time, lat, lon) float32 ...
    secmb    (time, lat, lon) float32 ...
    secma    (time, lat, lon) float32 ...
Attributes: (12/25)
    CDI:                     Climate Data Interface version 1.9.10 (https://m...
    Conventions:             CF-1.6
    institution:             University of Maryland (UofMD), College Park, MD...
    host:                    UMD College Park
    contact:                 gchurtt@umd.edu, lchini@umd.edu, steve.frolking@...
    creation_date:           2022-08-04T11:07:54Z
    ...                      ...
    title:                   UofMD LUH2-GCB2022 dataset prepared for Global C...
    activity_id:             Global Carbon Budget
    dataset_version_number:  LUH2-GCB2022
    source_id:               UofMD-landState-LUH2-GCB2022
    target_mip:              Global Carbon Budget
    references:              Hurtt et al. 2020, Chini et al. 2021

In [8]:
nbp = ['branches', 'stems', 'leaves', 'roots', 'litter', 'humus', 'charcoal', 'short lifetime timber', 'long lifetime timber']

# %%
# # Agriculture (GLCT 1)
agri_branch_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
agri_stems_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
agri_leaves_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
agri_roots_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
agri_litter_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
agri_humus_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
agri_charcoal_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
agri_slt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
agri_llt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")

In [9]:
# # Extensive Grassland (GLCT 2)
ext_grass_branch_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
ext_grass_stems_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
ext_grass_leaves_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
ext_grass_roots_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
ext_grass_litter_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
ext_grass_humus_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
ext_grass_charcoal_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
ext_grass_slt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
ext_grass_llt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")


In [11]:

# # Carbon Plantation (GLCT 3)
carbon_plant_branch_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
carbon_plant_stems_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
carbon_plant_leaves_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
carbon_plant_roots_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
carbon_plant_litter_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
carbon_plant_humus_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
carbon_plant_charcoal_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
carbon_plant_slt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
carbon_plant_llt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")

# # Regrowth Forest Abandoning (GLCT 4)
rfa_branch_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rfa_stems_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rfa_leaves_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rfa_roots_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rfa_litter_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rfa_humus_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rfa_charcoal_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rfa_slt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rfa_llt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")

# # Regrowth Forest Timber (GLCT 5)
rft_branch_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rft_stems_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rft_leaves_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rft_roots_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rft_litter_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rft_humus_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rft_charcoal_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rft_slt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
rft_llt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")

# # Biofuels (GLCT 6)
biofuels_branch_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
biofuels_stems_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
biofuels_leaves_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
biofuels_roots_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
biofuels_litter_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
biofuels_humus_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
biofuels_charcoal_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
biofuels_slt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
biofuels_llt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")

# # Ice (GLCT 7)
#ice_branch_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
#ice_stems_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
#ice_leaves_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
#ice_roots_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
#ice_litter_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
#ice_humus_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
#ice_charcoal_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
#ice_slt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
#ice_llt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")

# # Tundra (GLCT 8)
tundra_branch_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
tundra_stems_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
tundra_leaves_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
tundra_roots_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
tundra_litter_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
tundra_humus_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
tundra_charcoal_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
tundra_slt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
tundra_llt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")

# # Wooded Tundra (GLCT 9)
wood_tundra_branch_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
wood_tundra_stems_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
wood_tundra_leaves_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
wood_tundra_roots_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
wood_tundra_litter_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
wood_tundra_humus_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
wood_tundra_charcoal_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
wood_tundra_slt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
wood_tundra_llt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")

# # Boreal Forest (GLCT 10)
boreal_forest_branch_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
boreal_forest_stems_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
boreal_forest_leaves_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
boreal_forest_roots_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
boreal_forest_litter_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
boreal_forest_humus_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
boreal_forest_charcoal_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
boreal_forest_slt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")
boreal_forest_llt_gbiomass5min = np.zeros((5, 2160, 4320), dtype="float64")


MemoryError: Unable to allocate 356. MiB for an array with shape (5, 2160, 4320) and data type float64

In [6]:
# # Agriculture
import numpy as np

new_agri_branch_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_agri_stems_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_agri_leaves_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_agri_roots_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_agri_litter_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_agri_humus_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_agri_charcoal_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_agri_slt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_agri_llt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")

In [22]:

# # Extensive Grassland
new_ext_grass_branch_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_ext_grass_stems_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_ext_grass_leaves_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_ext_grass_roots_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_ext_grass_litter_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_ext_grass_humus_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_ext_grass_charcoal_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_ext_grass_slt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_ext_grass_llt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")

# # Carbon Plantation
new_carbon_plant_branch_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_carbon_plant_stems_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_carbon_plant_leaves_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_carbon_plant_roots_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_carbon_plant_litter_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_carbon_plant_humus_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_carbon_plant_charcoal_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_carbon_plant_slt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_carbon_plant_llt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")

# # Regrowth Forest Abandoning
new_rfa_branch_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rfa_stems_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rfa_leaves_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rfa_roots_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rfa_litter_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rfa_humus_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rfa_charcoal_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rfa_slt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rfa_llt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")

# # Regrowth Forest Timber
new_rft_branch_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rft_stems_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rft_leaves_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rft_roots_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rft_litter_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rft_humus_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rft_charcoal_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rft_slt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_rft_llt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")

# # Biofuels
new_biofuels_branch_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_biofuels_stems_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_biofuels_leaves_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_biofuels_roots_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_biofuels_litter_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_biofuels_humus_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_biofuels_charcoal_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_biofuels_slt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_biofuels_llt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")

# # Ice
# new_ice_branch_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
# new_ice_stems_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
# new_ice_leaves_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
# new_ice_roots_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
# new_ice_litter_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
# new_ice_humus_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
# new_ice_charcoal_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
# new_ice_slt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
# new_ice_llt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")

# # Tundra
new_tundra_branch_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_tundra_stems_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_tundra_leaves_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_tundra_roots_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_tundra_litter_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_tundra_humus_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_tundra_charcoal_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_tundra_slt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_tundra_llt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")

# # Wooded Tundra
new_wood_tundra_branch_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_wood_tundra_stems_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_wood_tundra_leaves_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_wood_tundra_roots_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_wood_tundra_litter_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_wood_tundra_humus_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_wood_tundra_charcoal_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_wood_tundra_slt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_wood_tundra_llt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")

# # Boreal Forest
new_boreal_forest_branch_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_boreal_forest_stems_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_boreal_forest_leaves_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_boreal_forest_roots_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_boreal_forest_litter_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_boreal_forest_humus_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_boreal_forest_charcoal_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_boreal_forest_slt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")
new_boreal_forest_llt_gbiomass5min = np.zeros((27, 720, 1440), dtype="float64")


MemoryError: Unable to allocate 214. MiB for an array with shape (27, 720, 1440) and data type float64

In [10]:
for i in range(27):
    slide_agri_branch = sliding_window_view(agri_branch_gbiomass5min[i], (3,3))[::3,::3]
    new_agri_branch_gbiomass5min[i] = slide_agri_branch.mean(axis=-1).mean(axis=-1)
    
    slide_agri_stems = sliding_window_view(agri_stems_gbiomass5min[i], (3,3))[::3,::3]
    new_agri_stems_gbiomass5min[i] = slide_agri_stems.mean(axis=-1).mean(axis=-1)
    
    slide_agri_leaves = sliding_window_view(agri_leaves_gbiomass5min[i], (3,3))[::3,::3]
    new_agri_leaves_gbiomass5min[i] = slide_agri_leaves.mean(axis=-1).mean(axis=-1)
    
    slide_agri_roots = sliding_window_view(agri_roots_gbiomass5min[i], (3,3))[::3,::3]
    new_agri_roots_gbiomass5min[i] = slide_agri_roots.mean(axis=-1).mean(axis=-1)
    
    slide_agri_humus = sliding_window_view(agri_humus_gbiomass5min[i], (3,3))[::3,::3]
    new_agri_humus_gbiomass5min[i] = slide_agri_humus.mean(axis=-1).mean(axis=-1)
    
    slide_agri_litter = sliding_window_view(agri_litter_gbiomass5min[i], (3,3))[::3,::3]
    new_agri_litter_gbiomass5min[i] = slide_agri_litter.mean(axis=-1).mean(axis=-1)
    
    slide_agri_charcoal = sliding_window_view(agri_charcoal_gbiomass5min[i], (3,3))[::3,::3]
    new_agri_charcoal_gbiomass5min[i] = slide_agri_charcoal.mean(axis=-1).mean(axis=-1)
    
    slide_agri_slt = sliding_window_view(agri_slt_gbiomass5min[i], (3,3))[::3,::3]
    new_agri_slt_gbiomass5min[i] = slide_agri_slt.mean(axis=-1).mean(axis=-1)
    
    slide_agri_llt = sliding_window_view(agri_llt_gbiomass5min[i], (3,3))[::3,::3]
    new_agri_llt_gbiomass5min[i] = slide_agri_llt.mean(axis=-1).mean(axis=-1)

MemoryError: Unable to allocate 23.7 MiB for an array with shape (720, 1440, 3) and data type float64

In [ ]:
# %%
for time in range(27):
        agri_branch_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==1,\
                                gbiomass_5min.sel(NBP=b'branches                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                agri_branch_gbiomass5min[time])
        agri_stems_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==1,\
                                gbiomass_5min.sel(NBP=b'stems                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                agri_stems_gbiomass5min[time])
        agri_leaves_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==1,\
                                gbiomass_5min.sel(NBP=b'leaves                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                               agri_leaves_gbiomass5min[time])
        agri_roots_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==1,\
                                gbiomass_5min.sel(NBP=b'roots                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                agri_roots_gbiomass5min[time])
        agri_humus_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==1,\
                                gbiomass_5min.sel(NBP=b'humus                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                agri_humus_gbiomass5min[time])
        agri_litter_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==1,\
                                gbiomass_5min.sel(NBP=b'litter                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                agri_litter_gbiomass5min[time])
        agri_charcoal_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==1,\
                                gbiomass_5min.sel(NBP=b'charcoal                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                agri_charcoal_gbiomass5min[time])
        agri_slt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==1,\
                                gbiomass_5min.sel(NBP=b'short lifetime timber                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                agri_slt_gbiomass5min[time])
        agri_llt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==1,\
                                gbiomass_5min.sel(NBP=b'long lifetime timber                              ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                agri_llt_gbiomass5min[time])

In [27]:
# %%
new_expand_agri_branch = np.expand_dims(new_agri_branch_gbiomass5min, axis=-1)
new_expand_agri_stems = np.expand_dims(new_agri_stems_gbiomass5min, axis=-1)
new_expand_agri_leaves = np.expand_dims(new_agri_leaves_gbiomass5min, axis=-1)
new_expand_agri_roots = np.expand_dims(new_agri_roots_gbiomass5min, axis=-1)
new_expand_agri_litter = np.expand_dims(new_agri_litter_gbiomass5min, axis=-1)
new_expand_agri_humus = np.expand_dims(new_agri_humus_gbiomass5min, axis=-1)
new_expand_agri_charcoal = np.expand_dims(new_agri_charcoal_gbiomass5min, axis=-1)
new_expand_agri_slt = np.expand_dims(new_agri_slt_gbiomass5min, axis=-1)
new_expand_agri_llt = np.expand_dims(new_agri_llt_gbiomass5min, axis=-1)

# %%
new_agri_resample = np.concatenate([new_expand_agri_branch, new_expand_agri_stems, new_expand_agri_leaves,
                                    new_expand_agri_roots, new_expand_agri_litter, new_expand_agri_humus,
                                    new_expand_agri_charcoal, new_expand_agri_slt, new_expand_agri_llt], axis=-1)

# %%
agri_resample = xr.Dataset({
        "agriculture":(["time", "lat", "lon", "NBP"], new_agri_resample)
    },
    coords={
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy(),
        "time":pd.date_range("1970-01-01", "2100-01-01", freq='5YS'),
        "NBP":nbp
    })

# %%
agri_resample.to_netcdf("D:/kerja/asisten kerja/vol/milkunarc/cadlan/Analysis1_slidingwindows/GLCT1-AGRICULTURE-BIOMASS-RESAMPLE-1970-2100.nc", mode='w', format="NETCDF4")


RuntimeError: NetCDF: HDF error

In [ ]:

for time in range(27):
        ext_grass_branch_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==2,\
                                gbiomass_5min.sel(NBP=b'branches                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                ext_grass_branch_gbiomass5min[time]
                                )
        ext_grass_stems_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==2,\
                                gbiomass_5min.sel(NBP=b'stems                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                ext_grass_stems_gbiomass5min[time]
                                )
        ext_grass_leaves_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==2,\
                                gbiomass_5min.sel(NBP=b'leaves                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                ext_grass_leaves_gbiomass5min[time]
                                )
        ext_grass_roots_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==2,\
                                gbiomass_5min.sel(NBP=b'roots                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                ext_grass_roots_gbiomass5min[time]
                                )
        ext_grass_humus_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==2,\
                                gbiomass_5min.sel(NBP=b'humus                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                ext_grass_humus_gbiomass5min[time]
                                )
        ext_grass_litter_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==2,\
                                gbiomass_5min.sel(NBP=b'litter                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                ext_grass_litter_gbiomass5min[time]
                                )
        ext_grass_charcoal_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==2,\
                                gbiomass_5min.sel(NBP=b'charcoal                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                ext_grass_charcoal_gbiomass5min[time]
                                )
        ext_grass_slt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==2,\
                                gbiomass_5min.sel(NBP=b'short lifetime timber                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                ext_grass_slt_gbiomass5min[time]
                                )
        ext_grass_llt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==2,\
                                gbiomass_5min.sel(NBP=b'long lifetime timber                              ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                ext_grass_llt_gbiomass5min[time]
                                )

In [ ]:
# %%
for i in range(27):
    slide_ext_grass_branch = sliding_window_view(ext_grass_branch_gbiomass5min[i], (3,3))[::3,::3]
    new_ext_grass_branch_gbiomass5min[i] = slide_ext_grass_branch.mean(axis=-1).mean(axis=-1)
    
    slide_ext_grass_stems = sliding_window_view(ext_grass_stems_gbiomass5min[i], (3,3))[::3,::3]
    new_ext_grass_stems_gbiomass5min[i] = slide_ext_grass_stems.mean(axis=-1).mean(axis=-1)
    
    slide_ext_grass_leaves = sliding_window_view(ext_grass_leaves_gbiomass5min[i], (3,3))[::3,::3]
    new_ext_grass_leaves_gbiomass5min[i] = slide_ext_grass_leaves.mean(axis=-1).mean(axis=-1)
    
    slide_ext_grass_roots = sliding_window_view(ext_grass_roots_gbiomass5min[i], (3,3))[::3,::3]
    new_ext_grass_roots_gbiomass5min[i] = slide_ext_grass_roots.mean(axis=-1).mean(axis=-1)
    
    slide_ext_grass_humus = sliding_window_view(ext_grass_humus_gbiomass5min[i], (3,3))[::3,::3]
    new_ext_grass_humus_gbiomass5min[i] = slide_ext_grass_humus.mean(axis=-1).mean(axis=-1)
    
    slide_ext_grass_litter = sliding_window_view(ext_grass_litter_gbiomass5min[i], (3,3))[::3,::3]
    new_ext_grass_litter_gbiomass5min[i] = slide_ext_grass_litter.mean(axis=-1).mean(axis=-1)
    
    slide_ext_grass_charcoal = sliding_window_view(ext_grass_charcoal_gbiomass5min[i], (3,3))[::3,::3]
    new_ext_grass_charcoal_gbiomass5min[i] = slide_ext_grass_charcoal.mean(axis=-1).mean(axis=-1)
    
    slide_ext_grass_slt = sliding_window_view(ext_grass_slt_gbiomass5min[i], (3,3))[::3,::3]
    new_ext_grass_slt_gbiomass5min[i] = slide_ext_grass_slt.mean(axis=-1).mean(axis=-1)
    
    slide_ext_grass_llt = sliding_window_view(ext_grass_llt_gbiomass5min[i], (3,3))[::3,::3]
    new_ext_grass_llt_gbiomass5min[i] = slide_ext_grass_llt.mean(axis=-1).mean(axis=-1)

# %%
new_expand_ext_grass_branch = np.expand_dims(new_ext_grass_branch_gbiomass5min, axis=-1)
new_expand_ext_grass_stems = np.expand_dims(new_ext_grass_stems_gbiomass5min, axis=-1)
new_expand_ext_grass_leaves = np.expand_dims(new_ext_grass_leaves_gbiomass5min, axis=-1)
new_expand_ext_grass_roots = np.expand_dims(new_ext_grass_roots_gbiomass5min, axis=-1)
new_expand_ext_grass_litter = np.expand_dims(new_ext_grass_litter_gbiomass5min, axis=-1)
new_expand_ext_grass_humus = np.expand_dims(new_ext_grass_humus_gbiomass5min, axis=-1)
new_expand_ext_grass_charcoal = np.expand_dims(new_ext_grass_charcoal_gbiomass5min, axis=-1)
new_expand_ext_grass_slt = np.expand_dims(new_ext_grass_slt_gbiomass5min, axis=-1)
new_expand_ext_grass_llt = np.expand_dims(new_ext_grass_llt_gbiomass5min, axis=-1)

# %%
new_ext_grass_resample = np.concatenate([new_expand_ext_grass_branch, new_expand_ext_grass_stems, new_expand_ext_grass_leaves,
                                    new_expand_ext_grass_roots, new_expand_ext_grass_litter, new_expand_ext_grass_humus,
                                    new_expand_ext_grass_charcoal, new_expand_ext_grass_slt, new_expand_ext_grass_llt], axis=-1)

# %%
new_ext_grass_resample.shape

# %%
ext_grass_resample = xr.Dataset({
        "extensive grassland":(["time", "lat", "lon", "NBP"], new_ext_grass_resample)
    },
    coords={
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy(),
        "time":pd.date_range("1970-01-01", "2100-01-01", freq='5YS'),
        "NBP":nbp
    })

# %%
ext_grass_resample.to_netcdf("D:/kerja/asisten kerja/vol/milkunarc/cadlan/Analysis1_slidingwindows/GLCT2-EXTENSIVE-GRASSLAND-BIOMASS-RESAMPLE-1970-2100.nc", mode='w', format="NETCDF4")


In [ ]:
# %% [markdown]
# ## GLCT 3 - Carbon Plantation

# %%
for time in range(27):
        carbon_plant_branch_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==3,\
                                gbiomass_5min.sel(NBP=b'branches                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                carbon_plant_branch_gbiomass5min[time]
                                )
        carbon_plant_stems_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==3,\
                                gbiomass_5min.sel(NBP=b'stems                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                carbon_plant_stems_gbiomass5min[time]
                                )
        carbon_plant_leaves_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==3,\
                                gbiomass_5min.sel(NBP=b'leaves                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                carbon_plant_leaves_gbiomass5min[time]
                                )
        carbon_plant_roots_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==3,\
                                gbiomass_5min.sel(NBP=b'roots                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                carbon_plant_roots_gbiomass5min[time]
                                )
        carbon_plant_humus_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==3,\
                                gbiomass_5min.sel(NBP=b'humus                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                carbon_plant_humus_gbiomass5min[time]
                                )
        carbon_plant_litter_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==3,\
                                gbiomass_5min.sel(NBP=b'litter                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                carbon_plant_litter_gbiomass5min[time]
                                )
        carbon_plant_charcoal_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==3,\
                                gbiomass_5min.sel(NBP=b'charcoal                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                carbon_plant_charcoal_gbiomass5min[time]
                                )
        carbon_plant_slt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==3,\
                                gbiomass_5min.sel(NBP=b'short lifetime timber                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                carbon_plant_slt_gbiomass5min[time]
                                )
        carbon_plant_llt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==3,\
                                gbiomass_5min.sel(NBP=b'long lifetime timber                              ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                carbon_plant_llt_gbiomass5min[time]
                                )

# %%
for i in range(27):
    slide_carbon_plant_branch = sliding_window_view(carbon_plant_branch_gbiomass5min[i], (3,3))[::3,::3]
    new_carbon_plant_branch_gbiomass5min[i] = slide_carbon_plant_branch.mean(axis=-1).mean(axis=-1)
    
    slide_carbon_plant_stems = sliding_window_view(carbon_plant_stems_gbiomass5min[i], (3,3))[::3,::3]
    new_carbon_plant_stems_gbiomass5min[i] = slide_carbon_plant_stems.mean(axis=-1).mean(axis=-1)
    
    slide_carbon_plant_leaves = sliding_window_view(carbon_plant_leaves_gbiomass5min[i], (3,3))[::3,::3]
    new_carbon_plant_leaves_gbiomass5min[i] = slide_carbon_plant_leaves.mean(axis=-1).mean(axis=-1)
    
    slide_carbon_plant_roots = sliding_window_view(carbon_plant_roots_gbiomass5min[i], (3,3))[::3,::3]
    new_carbon_plant_roots_gbiomass5min[i] = slide_carbon_plant_roots.mean(axis=-1).mean(axis=-1)
    
    slide_carbon_plant_humus = sliding_window_view(carbon_plant_humus_gbiomass5min[i], (3,3))[::3,::3]
    new_carbon_plant_humus_gbiomass5min[i] = slide_carbon_plant_humus.mean(axis=-1).mean(axis=-1)
    
    slide_carbon_plant_litter = sliding_window_view(carbon_plant_litter_gbiomass5min[i], (3,3))[::3,::3]
    new_carbon_plant_litter_gbiomass5min[i] = slide_carbon_plant_litter.mean(axis=-1).mean(axis=-1)
    
    slide_carbon_plant_charcoal = sliding_window_view(carbon_plant_charcoal_gbiomass5min[i], (3,3))[::3,::3]
    new_carbon_plant_charcoal_gbiomass5min[i] = slide_carbon_plant_charcoal.mean(axis=-1).mean(axis=-1)
    
    slide_carbon_plant_slt = sliding_window_view(carbon_plant_slt_gbiomass5min[i], (3,3))[::3,::3]
    new_carbon_plant_slt_gbiomass5min[i] = slide_carbon_plant_slt.mean(axis=-1).mean(axis=-1)
    
    slide_carbon_plant_llt = sliding_window_view(carbon_plant_llt_gbiomass5min[i], (3,3))[::3,::3]
    new_carbon_plant_llt_gbiomass5min[i] = slide_carbon_plant_llt.mean(axis=-1).mean(axis=-1)

# %%
new_expand_carbon_plant_branch = np.expand_dims(new_carbon_plant_branch_gbiomass5min, axis=-1)
new_expand_carbon_plant_stems = np.expand_dims(new_carbon_plant_stems_gbiomass5min, axis=-1)
new_expand_carbon_plant_leaves = np.expand_dims(new_carbon_plant_leaves_gbiomass5min, axis=-1)
new_expand_carbon_plant_roots = np.expand_dims(new_carbon_plant_roots_gbiomass5min, axis=-1)
new_expand_carbon_plant_litter = np.expand_dims(new_carbon_plant_litter_gbiomass5min, axis=-1)
new_expand_carbon_plant_humus = np.expand_dims(new_carbon_plant_humus_gbiomass5min, axis=-1)
new_expand_carbon_plant_charcoal = np.expand_dims(new_carbon_plant_charcoal_gbiomass5min, axis=-1)
new_expand_carbon_plant_slt = np.expand_dims(new_carbon_plant_slt_gbiomass5min, axis=-1)
new_expand_carbon_plant_llt = np.expand_dims(new_carbon_plant_llt_gbiomass5min, axis=-1)

# %%
new_carbon_plant_resample = np.concatenate([new_expand_carbon_plant_branch, new_expand_carbon_plant_stems, new_expand_carbon_plant_leaves,
                                        new_expand_carbon_plant_roots, new_expand_carbon_plant_litter, new_expand_carbon_plant_humus,
                                        new_expand_carbon_plant_charcoal, new_expand_carbon_plant_slt, new_expand_carbon_plant_llt], axis=-1)

# %%
new_carbon_plant_resample.shape

# %%
carbon_plant_resample = xr.Dataset({
        "carbon plantation":(["time", "lat", "lon", "NBP"], new_carbon_plant_resample)
    },
    coords={
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy(),
        "time":pd.date_range("1970-01-01", "2100-01-01", freq='5YS'),
        "NBP":nbp
    })

# %%
carbon_plant_resample.to_netcdf("D:/kerja/asisten kerja/vol/milkunarc/cadlan/Analysis1_slidingwindows/GLCT3-CARBON-PLANTATION-BIOMASS-RESAMPLE-1970-2100.nc", mode='w', format="NETCDF4")


In [ ]:
# %% [markdown]
# ## GLCT 4 - Regrowth Forest Abandoning

# %%
for time in range(27):
        rfa_branch_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==4,\
                                gbiomass_5min.sel(NBP=b'branches                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rfa_branch_gbiomass5min[time]
                                )
        rfa_stems_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==4,\
                                gbiomass_5min.sel(NBP=b'stems                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rfa_stems_gbiomass5min[time]
                                )
        rfa_leaves_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==4,\
                                gbiomass_5min.sel(NBP=b'leaves                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rfa_leaves_gbiomass5min[time]
                                )
        rfa_roots_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==4,\
                                gbiomass_5min.sel(NBP=b'roots                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rfa_roots_gbiomass5min[time]
                                )
        rfa_humus_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==4,\
                                gbiomass_5min.sel(NBP=b'humus                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rfa_humus_gbiomass5min[time]
                                )
        rfa_litter_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==4,\
                                gbiomass_5min.sel(NBP=b'litter                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rfa_litter_gbiomass5min[time]
                                )
        rfa_charcoal_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==4,\
                                gbiomass_5min.sel(NBP=b'charcoal                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rfa_charcoal_gbiomass5min[time]
                                )
        rfa_slt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==4,\
                                gbiomass_5min.sel(NBP=b'short lifetime timber                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rfa_slt_gbiomass5min[time]
                                )
        rfa_llt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==4,\
                                gbiomass_5min.sel(NBP=b'long lifetime timber                              ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rfa_llt_gbiomass5min[time]
                                )

# %%
for i in range(27):
    slide_rfa_branch = sliding_window_view(rfa_branch_gbiomass5min[i], (3,3))[::3,::3]
    new_rfa_branch_gbiomass5min[i] = slide_rfa_branch.mean(axis=-1).mean(axis=-1)
    
    slide_rfa_stems = sliding_window_view(rfa_stems_gbiomass5min[i], (3,3))[::3,::3]
    new_rfa_stems_gbiomass5min[i] = slide_rfa_stems.mean(axis=-1).mean(axis=-1)
    
    slide_rfa_leaves = sliding_window_view(rfa_leaves_gbiomass5min[i], (3,3))[::3,::3]
    new_rfa_leaves_gbiomass5min[i] = slide_rfa_leaves.mean(axis=-1).mean(axis=-1)
    
    slide_rfa_roots = sliding_window_view(rfa_roots_gbiomass5min[i], (3,3))[::3,::3]
    new_rfa_roots_gbiomass5min[i] = slide_rfa_roots.mean(axis=-1).mean(axis=-1)
    
    slide_rfa_humus = sliding_window_view(rfa_humus_gbiomass5min[i], (3,3))[::3,::3]
    new_rfa_humus_gbiomass5min[i] = slide_rfa_humus.mean(axis=-1).mean(axis=-1)
    
    slide_rfa_litter = sliding_window_view(rfa_litter_gbiomass5min[i], (3,3))[::3,::3]
    new_rfa_litter_gbiomass5min[i] = slide_rfa_litter.mean(axis=-1).mean(axis=-1)
    
    slide_rfa_charcoal = sliding_window_view(rfa_charcoal_gbiomass5min[i], (3,3))[::3,::3]
    new_rfa_charcoal_gbiomass5min[i] = slide_rfa_charcoal.mean(axis=-1).mean(axis=-1)
    
    slide_rfa_slt = sliding_window_view(rfa_slt_gbiomass5min[i], (3,3))[::3,::3]
    new_rfa_slt_gbiomass5min[i] = slide_rfa_slt.mean(axis=-1).mean(axis=-1)
    
    slide_rfa_llt = sliding_window_view(rfa_llt_gbiomass5min[i], (3,3))[::3,::3]
    new_rfa_llt_gbiomass5min[i] = slide_rfa_llt.mean(axis=-1).mean(axis=-1)

# %%
new_expand_rfa_branch = np.expand_dims(new_rfa_branch_gbiomass5min, axis=-1)
new_expand_rfa_stems = np.expand_dims(new_rfa_stems_gbiomass5min, axis=-1)
new_expand_rfa_leaves = np.expand_dims(new_rfa_leaves_gbiomass5min, axis=-1)
new_expand_rfa_roots = np.expand_dims(new_rfa_roots_gbiomass5min, axis=-1)
new_expand_rfa_litter = np.expand_dims(new_rfa_litter_gbiomass5min, axis=-1)
new_expand_rfa_humus = np.expand_dims(new_rfa_humus_gbiomass5min, axis=-1)
new_expand_rfa_charcoal = np.expand_dims(new_rfa_charcoal_gbiomass5min, axis=-1)
new_expand_rfa_slt = np.expand_dims(new_rfa_slt_gbiomass5min, axis=-1)
new_expand_rfa_llt = np.expand_dims(new_rfa_llt_gbiomass5min, axis=-1)

# %%
new_rfa_resample = np.concatenate([new_expand_rfa_branch, new_expand_rfa_stems, new_expand_rfa_leaves,
                                    new_expand_rfa_roots, new_expand_rfa_litter, new_expand_rfa_humus,
                                    new_expand_rfa_charcoal, new_expand_rfa_slt, new_expand_rfa_llt], axis=-1)

# %%
rfa_resample = xr.Dataset({
        "regrowth forest abandoning":(["time", "lat", "lon", "NBP"], new_rfa_resample)
    },
    coords={
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy(),
        "time":pd.date_range("1970-01-01", "2100-01-01", freq='5YS'),
        "NBP":nbp
    })

# %%
rfa_resample.to_netcdf("D:/kerja/asisten kerja/vol/milkunarc/cadlan/Analysis1_slidingwindows/GLCT4-REGROWTH-FOREST-ABANDONING-BIOMASS-RESAMPLE-1970-2100.nc", mode='w', format="NETCDF4")


In [ ]:
# %% [markdown]
# ## GLCT 5 - Regrowth Forest Timber

# %%
for time in range(27):
        rft_branch_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==5,\
                                gbiomass_5min.sel(NBP=b'branches                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rft_branch_gbiomass5min[time]
                                )
        rft_stems_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==5,\
                                gbiomass_5min.sel(NBP=b'stems                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rft_stems_gbiomass5min[time]
                                )
        rft_leaves_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==5,\
                                gbiomass_5min.sel(NBP=b'leaves                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rft_leaves_gbiomass5min[time]
                                )
        rft_roots_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==5,\
                                gbiomass_5min.sel(NBP=b'roots                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rft_roots_gbiomass5min[time]
                                )
        rft_humus_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==5,\
                                gbiomass_5min.sel(NBP=b'humus                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rft_humus_gbiomass5min[time]
                                )
        rft_litter_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==5,\
                                gbiomass_5min.sel(NBP=b'litter                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rft_litter_gbiomass5min[time]
                                )
        rft_charcoal_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==5,\
                                gbiomass_5min.sel(NBP=b'charcoal                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rft_charcoal_gbiomass5min[time]
                                )
        rft_slt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==5,\
                                gbiomass_5min.sel(NBP=b'short lifetime timber                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rft_slt_gbiomass5min[time]
                                )
        rft_llt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==5,\
                                gbiomass_5min.sel(NBP=b'long lifetime timber                              ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                rft_llt_gbiomass5min[time]
                                )

# %%
for i in range(27):
    slide_rft_branch = sliding_window_view(rft_branch_gbiomass5min[i], (3,3))[::3,::3]
    new_rft_branch_gbiomass5min[i] = slide_rft_branch.mean(axis=-1).mean(axis=-1)
    
    slide_rft_stems = sliding_window_view(rft_stems_gbiomass5min[i], (3,3))[::3,::3]
    new_rft_stems_gbiomass5min[i] = slide_rft_stems.mean(axis=-1).mean(axis=-1)
    
    slide_rft_leaves = sliding_window_view(rft_leaves_gbiomass5min[i], (3,3))[::3,::3]
    new_rft_leaves_gbiomass5min[i] = slide_rft_leaves.mean(axis=-1).mean(axis=-1)
    
    slide_rft_roots = sliding_window_view(rft_roots_gbiomass5min[i], (3,3))[::3,::3]
    new_rft_roots_gbiomass5min[i] = slide_rft_roots.mean(axis=-1).mean(axis=-1)
    
    slide_rft_humus = sliding_window_view(rft_humus_gbiomass5min[i], (3,3))[::3,::3]
    new_rft_humus_gbiomass5min[i] = slide_rft_humus.mean(axis=-1).mean(axis=-1)
    
    slide_rft_litter = sliding_window_view(rft_litter_gbiomass5min[i], (3,3))[::3,::3]
    new_rft_litter_gbiomass5min[i] = slide_rft_litter.mean(axis=-1).mean(axis=-1)
    
    slide_rft_charcoal = sliding_window_view(rft_charcoal_gbiomass5min[i], (3,3))[::3,::3]
    new_rft_charcoal_gbiomass5min[i] = slide_rft_charcoal.mean(axis=-1).mean(axis=-1)
    
    slide_rft_slt = sliding_window_view(rft_slt_gbiomass5min[i], (3,3))[::3,::3]
    new_rft_slt_gbiomass5min[i] = slide_rft_slt.mean(axis=-1).mean(axis=-1)
    
    slide_rft_llt = sliding_window_view(rft_llt_gbiomass5min[i], (3,3))[::3,::3]
    new_rft_llt_gbiomass5min[i] = slide_rft_llt.mean(axis=-1).mean(axis=-1)

# %%
new_expand_rft_branch = np.expand_dims(new_rft_branch_gbiomass5min, axis=-1)
new_expand_rft_stems = np.expand_dims(new_rft_stems_gbiomass5min, axis=-1)
new_expand_rft_leaves = np.expand_dims(new_rft_leaves_gbiomass5min, axis=-1)
new_expand_rft_roots = np.expand_dims(new_rft_roots_gbiomass5min, axis=-1)
new_expand_rft_litter = np.expand_dims(new_rft_litter_gbiomass5min, axis=-1)
new_expand_rft_humus = np.expand_dims(new_rft_humus_gbiomass5min, axis=-1)
new_expand_rft_charcoal = np.expand_dims(new_rft_charcoal_gbiomass5min, axis=-1)
new_expand_rft_slt = np.expand_dims(new_rft_slt_gbiomass5min, axis=-1)
new_expand_rft_llt = np.expand_dims(new_rft_llt_gbiomass5min, axis=-1)

# %%
new_rft_resample = np.concatenate([new_expand_rft_branch, new_expand_rft_stems, new_expand_rft_leaves,
                                    new_expand_rft_roots, new_expand_rft_litter, new_expand_rft_humus,
                                    new_expand_rft_charcoal, new_expand_rft_slt, new_expand_rft_llt], axis=-1)

# %%
rft_resample = xr.Dataset({
        "regrowth forest timber":(["time", "lat", "lon", "NBP"], new_rft_resample)
    },
    coords={
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy(),
        "time":pd.date_range("1970-01-01", "2100-01-01", freq='5YS'),
        "NBP":nbp
    })

# %%
rft_resample.to_netcdf("D:/kerja/asisten kerja/vol/milkunarc/cadlan/Analysis1_slidingwindows/GLCT5-REGROWTH-FOREST-TIMBER-BIOMASS-RESAMPLE-1970-2100.nc", mode='w', format="NETCDF4")


In [ ]:
# ## GLCT 6 - Biofuels

# %%
for time in range(27):
        biofuels_branch_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==6,\
                                gbiomass_5min.sel(NBP=b'branches                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                biofuels_branch_gbiomass5min[time]
                                )
        biofuels_stems_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==6,\
                                gbiomass_5min.sel(NBP=b'stems                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                biofuels_stems_gbiomass5min[time]
                                )
        biofuels_leaves_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==6,\
                                gbiomass_5min.sel(NBP=b'leaves                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                biofuels_leaves_gbiomass5min[time]
                                )
        biofuels_roots_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==6,\
                                gbiomass_5min.sel(NBP=b'roots                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                biofuels_roots_gbiomass5min[time]
                                )
        biofuels_humus_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==6,\
                                gbiomass_5min.sel(NBP=b'humus                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                biofuels_humus_gbiomass5min[time]
                                )
        biofuels_litter_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==6,\
                                gbiomass_5min.sel(NBP=b'litter                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                biofuels_litter_gbiomass5min[time]
                                )
        biofuels_charcoal_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==6,\
                                gbiomass_5min.sel(NBP=b'charcoal                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                biofuels_charcoal_gbiomass5min[time]
                                )
        biofuels_slt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==6,\
                                gbiomass_5min.sel(NBP=b'short lifetime timber                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                biofuels_slt_gbiomass5min[time]
                                )
        biofuels_llt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==6,\
                                gbiomass_5min.sel(NBP=b'long lifetime timber                              ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                biofuels_llt_gbiomass5min[time]
                                )

# %%
for i in range(27):
    slide_biofuels_branch = sliding_window_view(biofuels_branch_gbiomass5min[i], (3,3))[::3,::3]
    new_biofuels_branch_gbiomass5min[i] = slide_biofuels_branch.mean(axis=-1).mean(axis=-1)
    
    slide_biofuels_stems = sliding_window_view(biofuels_stems_gbiomass5min[i], (3,3))[::3,::3]
    new_biofuels_stems_gbiomass5min[i] = slide_biofuels_stems.mean(axis=-1).mean(axis=-1)
    
    slide_biofuels_leaves = sliding_window_view(biofuels_leaves_gbiomass5min[i], (3,3))[::3,::3]
    new_biofuels_leaves_gbiomass5min[i] = slide_biofuels_leaves.mean(axis=-1).mean(axis=-1)
    
    slide_biofuels_roots = sliding_window_view(biofuels_roots_gbiomass5min[i], (3,3))[::3,::3]
    new_biofuels_roots_gbiomass5min[i] = slide_biofuels_roots.mean(axis=-1).mean(axis=-1)
    
    slide_biofuels_humus = sliding_window_view(biofuels_humus_gbiomass5min[i], (3,3))[::3,::3]
    new_biofuels_humus_gbiomass5min[i] = slide_biofuels_humus.mean(axis=-1).mean(axis=-1)
    
    slide_biofuels_litter = sliding_window_view(biofuels_litter_gbiomass5min[i], (3,3))[::3,::3]
    new_biofuels_litter_gbiomass5min[i] = slide_biofuels_litter.mean(axis=-1).mean(axis=-1)
    
    slide_biofuels_charcoal = sliding_window_view(biofuels_charcoal_gbiomass5min[i], (3,3))[::3,::3]
    new_biofuels_charcoal_gbiomass5min[i] = slide_biofuels_charcoal.mean(axis=-1).mean(axis=-1)
    
    slide_biofuels_slt = sliding_window_view(biofuels_slt_gbiomass5min[i], (3,3))[::3,::3]
    new_biofuels_slt_gbiomass5min[i] = slide_biofuels_slt.mean(axis=-1).mean(axis=-1)
    
    slide_biofuels_llt = sliding_window_view(biofuels_llt_gbiomass5min[i], (3,3))[::3,::3]
    new_biofuels_llt_gbiomass5min[i] = slide_biofuels_llt.mean(axis=-1).mean(axis=-1)

# %%
new_expand_biofuels_branch = np.expand_dims(new_biofuels_branch_gbiomass5min, axis=-1)
new_expand_biofuels_stems = np.expand_dims(new_biofuels_stems_gbiomass5min, axis=-1)
new_expand_biofuels_leaves = np.expand_dims(new_biofuels_leaves_gbiomass5min, axis=-1)
new_expand_biofuels_roots = np.expand_dims(new_biofuels_roots_gbiomass5min, axis=-1)
new_expand_biofuels_litter = np.expand_dims(new_biofuels_litter_gbiomass5min, axis=-1)
new_expand_biofuels_humus = np.expand_dims(new_biofuels_humus_gbiomass5min, axis=-1)
new_expand_biofuels_charcoal = np.expand_dims(new_biofuels_charcoal_gbiomass5min, axis=-1)
new_expand_biofuels_slt = np.expand_dims(new_biofuels_slt_gbiomass5min, axis=-1)
new_expand_biofuels_llt = np.expand_dims(new_biofuels_llt_gbiomass5min, axis=-1)

# %%
new_biofuels_resample = np.concatenate([new_expand_biofuels_branch, new_expand_biofuels_stems, new_expand_biofuels_leaves,
                                    new_expand_biofuels_roots, new_expand_biofuels_litter, new_expand_biofuels_humus,
                                    new_expand_biofuels_charcoal, new_expand_biofuels_slt, new_expand_biofuels_llt], axis=-1)

# %%
biofuels_resample = xr.Dataset({
        "biofuels":(["time", "lat", "lon", "NBP"], new_biofuels_resample)
    },
    coords={
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy(),
        "time":pd.date_range("1970-01-01", "2100-01-01", freq='5YS'),
        "NBP":nbp
    })

# %%
biofuels_resample.to_netcdf("D:/kerja/asisten kerja/vol/milkunarc/cadlan/Analysis1_slidingwindows/GLCT6-BIOFUELS-BIOMASS-RESAMPLE-1970-2100.nc", mode='w', format="NETCDF4")


In [ ]:
# ## GLCT 8 - Tundra

# %%
for time in range(27):
        tundra_branch_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==8,\
                                gbiomass_5min.sel(NBP=b'branches                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                tundra_branch_gbiomass5min[time]
                                )
        tundra_stems_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==8,\
                                gbiomass_5min.sel(NBP=b'stems                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                tundra_stems_gbiomass5min[time]
                                )
        tundra_leaves_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==8,\
                                gbiomass_5min.sel(NBP=b'leaves                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                tundra_leaves_gbiomass5min[time]
                                )
        tundra_roots_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==8,\
                                gbiomass_5min.sel(NBP=b'roots                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                tundra_roots_gbiomass5min[time]
                                )
        tundra_humus_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==8,\
                                gbiomass_5min.sel(NBP=b'humus                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                tundra_humus_gbiomass5min[time]
                                )
        tundra_litter_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==8,\
                                gbiomass_5min.sel(NBP=b'litter                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                tundra_litter_gbiomass5min[time]
                                )
        tundra_charcoal_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==8,\
                                gbiomass_5min.sel(NBP=b'charcoal                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                tundra_charcoal_gbiomass5min[time]
                                )
        tundra_slt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==8,\
                                gbiomass_5min.sel(NBP=b'short lifetime timber                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                tundra_slt_gbiomass5min[time]
                                )
        tundra_llt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==8,\
                                gbiomass_5min.sel(NBP=b'long lifetime timber                              ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                tundra_llt_gbiomass5min[time]
                                )

# %%
for i in range(27):
    slide_tundra_branch = sliding_window_view(tundra_branch_gbiomass5min[i], (3,3))[::3,::3]
    new_tundra_branch_gbiomass5min[i] = slide_tundra_branch.mean(axis=-1).mean(axis=-1)
    
    slide_tundra_stems = sliding_window_view(tundra_stems_gbiomass5min[i], (3,3))[::3,::3]
    new_tundra_stems_gbiomass5min[i] = slide_tundra_stems.mean(axis=-1).mean(axis=-1)
    
    slide_tundra_leaves = sliding_window_view(tundra_leaves_gbiomass5min[i], (3,3))[::3,::3]
    new_tundra_leaves_gbiomass5min[i] = slide_tundra_leaves.mean(axis=-1).mean(axis=-1)
    
    slide_tundra_roots = sliding_window_view(tundra_roots_gbiomass5min[i], (3,3))[::3,::3]
    new_tundra_roots_gbiomass5min[i] = slide_tundra_roots.mean(axis=-1).mean(axis=-1)
    
    slide_tundra_humus = sliding_window_view(tundra_humus_gbiomass5min[i], (3,3))[::3,::3]
    new_tundra_humus_gbiomass5min[i] = slide_tundra_humus.mean(axis=-1).mean(axis=-1)
    
    slide_tundra_litter = sliding_window_view(tundra_litter_gbiomass5min[i], (3,3))[::3,::3]
    new_tundra_litter_gbiomass5min[i] = slide_tundra_litter.mean(axis=-1).mean(axis=-1)
    
    slide_tundra_charcoal = sliding_window_view(tundra_charcoal_gbiomass5min[i], (3,3))[::3,::3]
    new_tundra_charcoal_gbiomass5min[i] = slide_tundra_charcoal.mean(axis=-1).mean(axis=-1)
    
    slide_tundra_slt = sliding_window_view(tundra_slt_gbiomass5min[i], (3,3))[::3,::3]
    new_tundra_slt_gbiomass5min[i] = slide_tundra_slt.mean(axis=-1).mean(axis=-1)
    
    slide_tundra_llt = sliding_window_view(tundra_llt_gbiomass5min[i], (3,3))[::3,::3]
    new_tundra_llt_gbiomass5min[i] = slide_tundra_llt.mean(axis=-1).mean(axis=-1)

# %%
new_expand_tundra_branch = np.expand_dims(new_tundra_branch_gbiomass5min, axis=-1)
new_expand_tundra_stems = np.expand_dims(new_tundra_stems_gbiomass5min, axis=-1)
new_expand_tundra_leaves = np.expand_dims(new_tundra_leaves_gbiomass5min, axis=-1)
new_expand_tundra_roots = np.expand_dims(new_tundra_roots_gbiomass5min, axis=-1)
new_expand_tundra_litter = np.expand_dims(new_tundra_litter_gbiomass5min, axis=-1)
new_expand_tundra_humus = np.expand_dims(new_tundra_humus_gbiomass5min, axis=-1)
new_expand_tundra_charcoal = np.expand_dims(new_tundra_charcoal_gbiomass5min, axis=-1)
new_expand_tundra_slt = np.expand_dims(new_tundra_slt_gbiomass5min, axis=-1)
new_expand_tundra_llt = np.expand_dims(new_tundra_llt_gbiomass5min, axis=-1)

# %%
new_tundra_resample = np.concatenate([new_expand_tundra_branch, new_expand_tundra_stems, new_expand_tundra_leaves,
                                    new_expand_tundra_roots, new_expand_tundra_litter, new_expand_tundra_humus,
                                    new_expand_tundra_charcoal, new_expand_tundra_slt, new_expand_tundra_llt], axis=-1)

# %%
tundra_resample = xr.Dataset({
        "tundra":(["time", "lat", "lon", "NBP"], new_tundra_resample)
    },
    coords={
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy(),
        "time":pd.date_range("1970-01-01", "2100-01-01", freq='5YS'),
        "NBP":nbp
    })

# %%
tundra_resample.to_netcdf("D:/kerja/asisten kerja/vol/milkunarc/cadlan/Analysis1_slidingwindows/GLCT8-TUNDRA-BIOMASS-RESAMPLE-1970-2100.nc", mode='w', format="NETCDF4")

In [ ]:
# ## GLCT 9 - Wooded Tundra

# %%
for time in range(27):
        wood_tundra_branch_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==9,\
                                gbiomass_5min.sel(NBP=b'branches                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                wood_tundra_branch_gbiomass5min[time]
                                )
        wood_tundra_stems_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==9,\
                                gbiomass_5min.sel(NBP=b'stems                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                wood_tundra_stems_gbiomass5min[time]
                                )
        wood_tundra_leaves_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==9,\
                                gbiomass_5min.sel(NBP=b'leaves                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                wood_tundra_leaves_gbiomass5min[time]
                                )
        wood_tundra_roots_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==9,\
                                gbiomass_5min.sel(NBP=b'roots                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                wood_tundra_roots_gbiomass5min[time]
                                )
        wood_tundra_humus_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==9,\
                                gbiomass_5min.sel(NBP=b'humus                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                wood_tundra_humus_gbiomass5min[time]
                                )
        wood_tundra_litter_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==9,\
                                gbiomass_5min.sel(NBP=b'litter                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                wood_tundra_litter_gbiomass5min[time]
                                )
        wood_tundra_charcoal_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==9,\
                                gbiomass_5min.sel(NBP=b'charcoal                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                wood_tundra_charcoal_gbiomass5min[time]
                                )
        wood_tundra_slt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==9,\
                                gbiomass_5min.sel(NBP=b'short lifetime timber                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                wood_tundra_slt_gbiomass5min[time]
                                )
        wood_tundra_llt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==9,\
                                gbiomass_5min.sel(NBP=b'long lifetime timber                              ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                wood_tundra_llt_gbiomass5min[time]
                                )

# %%
for i in range(27):
    slide_wood_tundra_branch = sliding_window_view(wood_tundra_branch_gbiomass5min[i], (3,3))[::3,::3]
    new_wood_tundra_branch_gbiomass5min[i] = slide_wood_tundra_branch.mean(axis=-1).mean(axis=-1)
    
    slide_wood_tundra_stems = sliding_window_view(wood_tundra_stems_gbiomass5min[i], (3,3))[::3,::3]
    new_wood_tundra_stems_gbiomass5min[i] = slide_wood_tundra_stems.mean(axis=-1).mean(axis=-1)
    
    slide_wood_tundra_leaves = sliding_window_view(wood_tundra_leaves_gbiomass5min[i], (3,3))[::3,::3]
    new_wood_tundra_leaves_gbiomass5min[i] = slide_wood_tundra_leaves.mean(axis=-1).mean(axis=-1)
    
    slide_wood_tundra_roots = sliding_window_view(wood_tundra_roots_gbiomass5min[i], (3,3))[::3,::3]
    new_wood_tundra_roots_gbiomass5min[i] = slide_wood_tundra_roots.mean(axis=-1).mean(axis=-1)
    
    slide_wood_tundra_humus = sliding_window_view(wood_tundra_humus_gbiomass5min[i], (3,3))[::3,::3]
    new_wood_tundra_humus_gbiomass5min[i] = slide_wood_tundra_humus.mean(axis=-1).mean(axis=-1)
    
    slide_wood_tundra_litter = sliding_window_view(wood_tundra_litter_gbiomass5min[i], (3,3))[::3,::3]
    new_wood_tundra_litter_gbiomass5min[i] = slide_wood_tundra_litter.mean(axis=-1).mean(axis=-1)
    
    slide_wood_tundra_charcoal = sliding_window_view(wood_tundra_charcoal_gbiomass5min[i], (3,3))[::3,::3]
    new_wood_tundra_charcoal_gbiomass5min[i] = slide_wood_tundra_charcoal.mean(axis=-1).mean(axis=-1)
    
    slide_wood_tundra_slt = sliding_window_view(wood_tundra_slt_gbiomass5min[i], (3,3))[::3,::3]
    new_wood_tundra_slt_gbiomass5min[i] = slide_wood_tundra_slt.mean(axis=-1).mean(axis=-1)
    
    slide_wood_tundra_llt = sliding_window_view(wood_tundra_llt_gbiomass5min[i], (3,3))[::3,::3]
    new_wood_tundra_llt_gbiomass5min[i] = slide_wood_tundra_llt.mean(axis=-1).mean(axis=-1)

# %%
new_expand_wood_tundra_branch = np.expand_dims(new_wood_tundra_branch_gbiomass5min, axis=-1)
new_expand_wood_tundra_stems = np.expand_dims(new_wood_tundra_stems_gbiomass5min, axis=-1)
new_expand_wood_tundra_leaves = np.expand_dims(new_wood_tundra_leaves_gbiomass5min, axis=-1)
new_expand_wood_tundra_roots = np.expand_dims(new_wood_tundra_roots_gbiomass5min, axis=-1)
new_expand_wood_tundra_litter = np.expand_dims(new_wood_tundra_litter_gbiomass5min, axis=-1)
new_expand_wood_tundra_humus = np.expand_dims(new_wood_tundra_humus_gbiomass5min, axis=-1)
new_expand_wood_tundra_charcoal = np.expand_dims(new_wood_tundra_charcoal_gbiomass5min, axis=-1)
new_expand_wood_tundra_slt = np.expand_dims(new_wood_tundra_slt_gbiomass5min, axis=-1)
new_expand_wood_tundra_llt = np.expand_dims(new_wood_tundra_llt_gbiomass5min, axis=-1)

# %%
new_wood_tundra_resample = np.concatenate([new_expand_wood_tundra_branch, new_expand_wood_tundra_stems, new_expand_wood_tundra_leaves,
                                    new_expand_wood_tundra_roots, new_expand_wood_tundra_litter, new_expand_wood_tundra_humus,
                                    new_expand_wood_tundra_charcoal, new_expand_wood_tundra_slt, new_expand_wood_tundra_llt], axis=-1)

# %%
wood_tundra_resample = xr.Dataset({
        "wooded tundra":(["time", "lat", "lon", "NBP"], new_wood_tundra_resample)
    },
    coords={
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy(),
        "time":pd.date_range("1970-01-01", "2100-01-01", freq='5YS'),
        "NBP":nbp
    })

# %%
wood_tundra_resample.to_netcdf("D:/kerja/asisten kerja/vol/milkunarc/cadlan/Analysis1_slidingwindows/GLCT9-WOODED-TUNDRA-BIOMASS-RESAMPLE-1970-2100.nc", mode='w', format="NETCDF4")


In [ ]:
 ## GLCT 10 - Boreal Forest

# %%
for time in range(27):
        boreal_forest_branch_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==10,\
                                gbiomass_5min.sel(NBP=b'branches                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                boreal_forest_branch_gbiomass5min[time]
                                )
        boreal_forest_stems_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==10,\
                                gbiomass_5min.sel(NBP=b'stems                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                boreal_forest_stems_gbiomass5min[time]
                                )
        boreal_forest_leaves_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==10,\
                                gbiomass_5min.sel(NBP=b'leaves                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                boreal_forest_leaves_gbiomass5min[time]
                                )
        boreal_forest_roots_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==10,\
                                gbiomass_5min.sel(NBP=b'roots                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                boreal_forest_roots_gbiomass5min[time]
                                )
        boreal_forest_humus_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==10,\
                                gbiomass_5min.sel(NBP=b'humus                                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                boreal_forest_humus_gbiomass5min[time]
                                )
        boreal_forest_litter_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==10,\
                                gbiomass_5min.sel(NBP=b'litter                                            ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                boreal_forest_litter_gbiomass5min[time]
                                )
        boreal_forest_charcoal_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==10,\
                                gbiomass_5min.sel(NBP=b'charcoal                                          ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                boreal_forest_charcoal_gbiomass5min[time]
                                )
        boreal_forest_slt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==10,\
                                gbiomass_5min.sel(NBP=b'short lifetime timber                             ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                boreal_forest_slt_gbiomass5min[time]
                                )
        boreal_forest_llt_gbiomass5min[time] = np.where(glct.isel(time=time)['GLCT'].to_numpy()==10,\
                                gbiomass_5min.sel(NBP=b'long lifetime timber                              ').isel(time=time)['GBIOMASS']\
                                        .to_numpy(),\
                                boreal_forest_llt_gbiomass5min[time]
                                )

# %%
for i in range(27):
    slide_boreal_forest_branch = sliding_window_view(boreal_forest_branch_gbiomass5min[i], (3,3))[::3,::3]
    new_boreal_forest_branch_gbiomass5min[i] = slide_boreal_forest_branch.mean(axis=-1).mean(axis=-1)
    
    slide_boreal_forest_stems = sliding_window_view(boreal_forest_stems_gbiomass5min[i], (3,3))[::3,::3]
    new_boreal_forest_stems_gbiomass5min[i] = slide_boreal_forest_stems.mean(axis=-1).mean(axis=-1)
    
    slide_boreal_forest_leaves = sliding_window_view(boreal_forest_leaves_gbiomass5min[i], (3,3))[::3,::3]
    new_boreal_forest_leaves_gbiomass5min[i] = slide_boreal_forest_leaves.mean(axis=-1).mean(axis=-1)
    
    slide_boreal_forest_roots = sliding_window_view(boreal_forest_roots_gbiomass5min[i], (3,3))[::3,::3]
    new_boreal_forest_roots_gbiomass5min[i] = slide_boreal_forest_roots.mean(axis=-1).mean(axis=-1)
    
    slide_boreal_forest_humus = sliding_window_view(boreal_forest_humus_gbiomass5min[i], (3,3))[::3,::3]
    new_boreal_forest_humus_gbiomass5min[i] = slide_boreal_forest_humus.mean(axis=-1).mean(axis=-1)
    
    slide_boreal_forest_litter = sliding_window_view(boreal_forest_litter_gbiomass5min[i], (3,3))[::3,::3]
    new_boreal_forest_litter_gbiomass5min[i] = slide_boreal_forest_litter.mean(axis=-1).mean(axis=-1)
    
    slide_boreal_forest_charcoal = sliding_window_view(boreal_forest_charcoal_gbiomass5min[i], (3,3))[::3,::3]
    new_boreal_forest_charcoal_gbiomass5min[i] = slide_boreal_forest_charcoal.mean(axis=-1).mean(axis=-1)
    
    slide_boreal_forest_slt = sliding_window_view(boreal_forest_slt_gbiomass5min[i], (3,3))[::3,::3]
    new_boreal_forest_slt_gbiomass5min[i] = slide_boreal_forest_slt.mean(axis=-1).mean(axis=-1)
    
    slide_boreal_forest_llt = sliding_window_view(boreal_forest_llt_gbiomass5min[i], (3,3))[::3,::3]
    new_boreal_forest_llt_gbiomass5min[i] = slide_boreal_forest_llt.mean(axis=-1).mean(axis=-1)

# %%
new_expand_boreal_forest_branch = np.expand_dims(new_boreal_forest_branch_gbiomass5min, axis=-1)
new_expand_boreal_forest_stems = np.expand_dims(new_boreal_forest_stems_gbiomass5min, axis=-1)
new_expand_boreal_forest_leaves = np.expand_dims(new_boreal_forest_leaves_gbiomass5min, axis=-1)
new_expand_boreal_forest_roots = np.expand_dims(new_boreal_forest_roots_gbiomass5min, axis=-1)
new_expand_boreal_forest_litter = np.expand_dims(new_boreal_forest_litter_gbiomass5min, axis=-1)
new_expand_boreal_forest_humus = np.expand_dims(new_boreal_forest_humus_gbiomass5min, axis=-1)
new_expand_boreal_forest_charcoal = np.expand_dims(new_boreal_forest_charcoal_gbiomass5min, axis=-1)
new_expand_boreal_forest_slt = np.expand_dims(new_boreal_forest_slt_gbiomass5min, axis=-1)
new_expand_boreal_forest_llt = np.expand_dims(new_boreal_forest_llt_gbiomass5min, axis=-1)

# %%
new_boreal_forest_resample = np.concatenate([new_expand_boreal_forest_branch, new_expand_boreal_forest_stems, new_expand_boreal_forest_leaves,
                                    new_expand_boreal_forest_roots, new_expand_boreal_forest_litter, new_expand_boreal_forest_humus,
                                    new_expand_boreal_forest_charcoal, new_expand_boreal_forest_slt, new_expand_boreal_forest_llt], axis=-1)

# %%
boreal_forest_resample = xr.Dataset({
        "boreal forest":(["time", "lat", "lon", "NBP"], new_boreal_forest_resample)
    },
    coords={
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy(),
        "time":pd.date_range("1970-01-01", "2100-01-01", freq='5YS'),
        "NBP":nbp
    })

# %%
boreal_forest_resample.to_netcdf("D:/kerja/asisten kerja/vol/milkunarc/cadlan/Analysis1_slidingwindows/GLCT10-BOREAL-FOREST-BIOMASS-RESAMPLE-1970-2100.nc", mode='w', format="NETCDF4")
